### BlackScholes delta hedging ###

A Python function which calculates the terminal profit or loss from a strategy which involves selling a European call or put option at time 0 and hedging it by a Black/Scholes delta-hedging strategy with rebalancing at discrete time points.

The function takes six arguments (in this order): 
 
S, K, sigma, r, T and a 1 for a call or -1 for a put. 

Note that here, S is a NumPy array of underlying stock prices

In [2]:
### a Python function which calculates the terminal profit or loss from a strategy which involves 
### selling a European call or put option at time 0 and hedging it by a Black/Scholes delta-hedging strategy 
### with rebalancing at discrete time points 
### The function takes six arguments (in this order): 
### S, K, sigma, r, T and a 1 for a call or -1 for a put. 
### Note that here, S is a NumPy array of underlying stock prices

import numpy as np
import pandas as pd
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

def BlackScholesHedgePath(S,K,sgm,r,T,callput):
    
    running_pnl = 0
    
    for i in range(1, len(S)):
        a = (T/(len(S)-1))*i
        d1 = (np.log(S[i] / K) + (r + ((sgm**2)/2))*((T-a))) / (sgm * (np.sqrt((T-a))))
        d2 = d1 - (sgm * (np.sqrt((T-a))))
        
        b = (T/(len(S)-1)) * (i-1)
        d3 = (np.log(S[i-1] / K) + (r + ((sgm**2)/2))*((T-b))) / (sgm * (np.sqrt((T-b))))
        d4 = d3 - (sgm * (np.sqrt((T-b))))
        
        if i < len(S):
            
            S_i = S[i] * (norm.cdf(d3) - norm.cdf(d1))
            K_i = K * np.exp(-r*(T-a)) * (norm.cdf(d4) - norm.cdf(d2))
            
            if callput == 1:
                pnl = (S_i - K_i)
            elif callput == -1:
                pnl = (K_i - S_i)
                
            running_pnl += (pnl * np.exp(r*(T-a)))
            
        elif i == len(S):
            S_i = S.iloc[i,1] * norm.cdf(d3)
            K_i = K * norm.cdf(d4)
            
            if callput == 1:
                pnl = (S_i - K_i - max(0, S[i] - K))
            elif callput == -1:
                pnl = (K_i - S_i - max(0,   K- S[i]))
                
            running_pnl += pnl
            
    return running_pnl


if __name__ == '__main__':
    stockdata = pd.read_csv('price_path.csv')
    stock = np.array(stockdata['S(t)'])
    strike = float(input('Enter the strike price: '))
    sigma = 0.3
    interest = 0.05
    maturity = 2
    callput = int(input('Enter 1 for call or -1 for put option: '))
    PnL = BlackScholesHedgePath(stock,strike,sigma,interest,maturity,callput)
    print('The profit/loss from the hedged option position is '+str(PnL))

Enter the strike price: 100
Enter 1 for call or -1 for put option: 1
The profit/loss from the hedged option position is 3.7001921391495163
